In [1]:
import pandas as pd
import sqlite3

In [2]:
file_path= r'C:\Users\MIHIR_KOSREKAR\OneDrive\Desktop\doing_business_data_analysis\P_Data_Extract_From_Doing_Business.xlsx'
original_df= pd.read_excel(file_path)
original_df.head()

,Country Name,Country Code,Series Name,Series Code,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019]
0,Singapore,SGP,Dealing with construction permits (DB06-15 met...,IC.CNST.PRMT.DFRN.DB0615,82.73673,82.74383,82.803,82.85477,82.87985,80.89801,..,..,..,..,..
1,Singapore,SGP,Dealing with construction permits (DB16-20 met...,IC.CNST.PRMT.DFRN.DB1619,..,..,..,..,..,80.67351,80.68634,83.74584,84.69304,84.73042,87.86398
2,Singapore,SGP,Dealing with construction permits: Building qu...,IC.DCP.BQC.XD.015.DB1619,..,..,..,..,..,12,12,12,12,12,13
3,Singapore,SGP,Dealing with construction permits: Building qu...,IC.CNST.PRMT.BQCI.015.DB1619.DFRN,..,..,..,..,..,80,80,80,80,80,86.66667
4,Singapore,SGP,Dealing with construction permits: Cost (% of ...,IC.CNST.PRMT.COST.WRH.VAL,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.3


In [3]:
original_df.columns.tolist()

['Country Name',
 'Country Code',
 'Series Name',
 'Series Code',
 '2009 [YR2009]',
 '2010 [YR2010]',
 '2011 [YR2011]',
 '2012 [YR2012]',
 '2013 [YR2013]',
 '2014 [YR2014]',
 '2015 [YR2015]',
 '2016 [YR2016]',
 '2017 [YR2017]',
 '2018 [YR2018]',
 '2019 [YR2019]']

In [4]:
conn= sqlite3.connect(':memory:')
original_df.to_sql('dbsn', conn, index= False, if_exists= 'replace') #'dbsn' is table name here, doing business, used for SQL queries

1940

In [5]:
# Creating function for SQL queries
def run_sql(query: str):
    return pd.DataFrame(pd.read_sql_query(query, conn))

In [6]:
country_df= run_sql('select distinct [Country Name], [Country Code] from dbsn')

In [7]:
country_df

,Country Name,Country Code
0,Singapore,SGP
1,United Kingdom,GBR
2,"Taiwan, China",TWN
3,India,IND
4,Indonesia,IDN
5,Spain,ESP
6,Australia,AUS
7,Poland,POL
8,Philippines,PHL
9,United States,USA


In [8]:
series_df= run_sql('select distinct [Series Name], [Series Code] from dbsn')

In [9]:
series_df

,Series Name,Series Code
0,Dealing with construction permits (DB06-15 met...,IC.CNST.PRMT.DFRN.DB0615
1,Dealing with construction permits (DB16-20 met...,IC.CNST.PRMT.DFRN.DB1619
2,Dealing with construction permits: Building qu...,IC.DCP.BQC.XD.015.DB1619
3,Dealing with construction permits: Building qu...,IC.CNST.PRMT.BQCI.015.DB1619.DFRN
4,Dealing with construction permits: Cost (% of ...,IC.CNST.PRMT.COST.WRH.VAL
...,...,...
189,Trading across borders: Time to export: Border...,TRD.ACRS.BRDR.EXPT.TM.BRDR.COMP.HR.DB1619.DFRN
190,Trading across borders: Time to export: Docume...,TRD.ACRS.BRDR.EXPT.TM.DOC.COMP.HR.DB1619.DFRN
191,Trading across borders: Time to import (days) ...,TRD.ACRS.BRDR.IMP.TM.DY.DB0615.DFRN
192,Trading across borders: Time to import: Border...,TRD.ACRS.BRDR.IMP.TM.BRDR.COMP.HR.DB1619.DFRN


In [10]:
melted_df= pd.melt(original_df,
                   id_vars= ['Country Name', 'Country Code', 'Series Name', 'Series Code'],
                   value_vars= [col for col in original_df.columns if col not in ['Country Name', 'Country Code', 'Series Name', 'Series Code']],
                   var_name= 'Year',
                   value_name= 'Value')

In [11]:
melted_df['Year']= melted_df['Year'].str[:4]
melted_df= melted_df[melted_df['Value'].str.contains(r'[0-9]', case= False, na= True)].reset_index(drop= True)

In [12]:
melted_df['Year']= melted_df['Year'].astype(int)
melted_df['Value']= melted_df['Value'].astype(int)

In [13]:
melted_df

,Country Name,Country Code,Series Name,Series Code,Year,Value
0,Singapore,SGP,Dealing with construction permits (DB06-15 met...,IC.CNST.PRMT.DFRN.DB0615,2009,82
1,Singapore,SGP,Dealing with construction permits: Cost (% of ...,IC.CNST.PRMT.COST.WRH.VAL,2009,3
2,Singapore,SGP,Dealing with construction permits: Cost (% of ...,IC.CNST.PRMT.COST.WRH.VAL.DFRN,2009,82
3,Singapore,SGP,Dealing with construction permits: Procedures ...,IC.CNST.PRMT.PROC.NO,2009,13
4,Singapore,SGP,Dealing with construction permits: Procedures ...,IC.CNST.PRMT.PROC.NO.DFRN,2009,68
...,...,...,...,...,...,...
13201,United States,USA,Trading across borders: Cost to import: Docume...,TRD.ACRS.BRDR.IMP.COST.DOC.COMP.CD.DB1619.DFRN,2019,85
13202,United States,USA,Trading across borders: Time to export: Border...,TRD.ACRS.BRDR.EXPT.TM.BRDR.COMP.HR.DB1619.DFRN,2019,99
13203,United States,USA,Trading across borders: Time to export: Docume...,TRD.ACRS.BRDR.EXPT.TM.DOC.COMP.HR.DB1619.DFRN,2019,99
13204,United States,USA,Trading across borders: Time to import: Border...,TRD.ACRS.BRDR.IMP.TM.BRDR.COMP.HR.DB1619.DFRN,2019,99


In [14]:
melted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13206 entries, 0 to 13205
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country Name  13206 non-null  object
 1   Country Code  13206 non-null  object
 2   Series Name   13206 non-null  object
 3   Series Code   13206 non-null  object
 4   Year          13206 non-null  int32 
 5   Value         13206 non-null  int32 
dtypes: int32(2), object(4)
memory usage: 516.0+ KB


In [15]:
melted_df.to_excel('cleaned_and_processed_data.xlsx')